In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event



In [6]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('../storage/track_ml_data/train_5.zip'):
    if cnt == 1:
        break
    cnt += 1

In [30]:
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm
from sklearn.cluster import DBSCAN

class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528]):                        
        self.rz_scales=rz_scales
    
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        xm = np.mean(x,axis=0)
        x = x - xm
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    def _init(self, dfh, w1, w2, w3, w4, w5, w6, w7, epsilon, Niter):
        dfh['r'] = np.sqrt(dfh['x'].values ** 2 + dfh['y'].values ** 2 + dfh['z'].values ** 2)
        dfh['rt'] = np.sqrt(dfh['x'].values ** 2 + dfh['y'].values ** 2)
        dfh['a0'] = np.arctan2(dfh['y'].values, dfh['x'].values)
        dfh['z1'] = dfh['z'].values / dfh['rt'].values
        dfh['z2'] = dfh['z'].values / dfh['r'].values
        dfh['s1'] = dfh['hit_id']
        dfh['N1'] = 1
        dfh['z1'] = dfh['z'].values / dfh['rt'].values
        dfh['z2'] = dfh['z'].values / dfh['r'].values
        dfh['x1'] = dfh['x'].values / dfh['y'].values
        dfh['x2'] = dfh['x'].values / dfh['r'].values
        dfh['x3'] = dfh['y'].values / dfh['r'].values
        dfh['x4'] = dfh['rt'].values / dfh['r'].values
        mm = 1
        result = []
        for ii in tqdm(range(int(Niter))):
            mm = mm * (-1)
            dfh['a1'] = dfh['a0'].values + mm * (dfh['rt'].values + 0.000005
                                                 * dfh['rt'].values ** 2) / 1000 * (ii / 2) / 180 * np.pi
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1', 'cosa1', 'z1', 'z2','x1','x2','x3','x4']].values)
            cx = np.array([w1, w1, w2, w3, w4, w5, w6, w7])
            dfs = np.multiply(dfs, cx)
            clusters = DBSCAN(eps=epsilon, min_samples=1, metric="euclidean", n_jobs=32).fit(dfs).labels_
            dfh['s2'] = clusters
            result += [clusters]
            dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
            maxs1 = dfh['s1'].max()
            dfh.loc[(dfh['N2'] > dfh['N1']) & (dfh['N2'] < 20),'s1'] = dfh['s2'] + maxs1
            dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            
        return dfh['s1'].values, result
    
    def predict(self, hits):         
        self.clusters, self.result  = self._init(hits,2.7474448671796874,1.3649721713529086,0.7034918842926337,
                                        0.0005549122352940002,0.023096034747190672,0.04619756315527515,
                                        0.2437077420144654,0.009750302717746615,338)
#         X = self._preprocess(hits) 
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                              metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
#         labels = np.unique(self.clusters)
#         self._eliminate_outliers(labels,X)          
#         max_len = np.max(self.clusters)
#         mask = self.clusters == 0
#         self.clusters[mask] = cl.fit_predict(X[mask])+max_len
        return self.clusters, self.result

In [31]:


model = Clusterer()
labels, result = model.predict(hits)



100%|██████████| 338/338 [05:00<00:00,  1.13it/s]


In [32]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [36]:
result

[array([    0,     1,     2, ..., 74107, 74108, 74102]),
 array([    0,     1,     2, ..., 74048, 12232, 74042]),
 array([    0,     1,     2, ..., 73942, 73943, 73934]),
 array([    0,     1,     2, ..., 73953, 73954, 73947]),
 array([    0,     1,     2, ..., 61962, 74017, 23036]),
 array([    0,     1,     2, ..., 74226, 18183, 60298]),
 array([    0,     1,     2, ..., 74092, 74093, 74084]),
 array([    0,     1,     2, ..., 74107, 74108, 74100]),
 array([    0,     1,     2, ..., 74053, 61865, 74046]),
 array([    0,     1,     2, ..., 74222, 74223, 74215]),
 array([    0,     1,     2, ..., 74096, 74097, 74089]),
 array([    0,     1,     2, ..., 74033, 74034, 74026]),
 array([    0,     1,     2, ..., 74079, 74080, 74073]),
 array([    0,     1,     2, ..., 74155, 74156, 61706]),
 array([    0,     1,     2, ..., 74317, 74318, 74310]),
 array([    0,     1,     2, ..., 74155, 74156, 44826]),
 array([    0,     1,     2, ..., 74495, 74496, 74487]),
 array([    0,     1,     2, ..

In [37]:
submission = create_one_event_submission(0, hits['hit_id'].values, labels)
score = score_event(truth, submission)
print(score)


0.5071140015567802


In [40]:
result

[array([    0,     1,     2, ..., 74107, 74108, 74102]),
 array([    0,     1,     2, ..., 74048, 12232, 74042]),
 array([    0,     1,     2, ..., 73942, 73943, 73934]),
 array([    0,     1,     2, ..., 73953, 73954, 73947]),
 array([    0,     1,     2, ..., 61962, 74017, 23036]),
 array([    0,     1,     2, ..., 74226, 18183, 60298]),
 array([    0,     1,     2, ..., 74092, 74093, 74084]),
 array([    0,     1,     2, ..., 74107, 74108, 74100]),
 array([    0,     1,     2, ..., 74053, 61865, 74046]),
 array([    0,     1,     2, ..., 74222, 74223, 74215]),
 array([    0,     1,     2, ..., 74096, 74097, 74089]),
 array([    0,     1,     2, ..., 74033, 74034, 74026]),
 array([    0,     1,     2, ..., 74079, 74080, 74073]),
 array([    0,     1,     2, ..., 74155, 74156, 61706]),
 array([    0,     1,     2, ..., 74317, 74318, 74310]),
 array([    0,     1,     2, ..., 74155, 74156, 44826]),
 array([    0,     1,     2, ..., 74495, 74496, 74487]),
 array([    0,     1,     2, ..

In [41]:
np.save('predicts/luis_all', result)

In [ ]:
0.5071140015567802
0.5022115428740757